In [2]:
import torch

In [7]:
def _combine(xs_b, ys_b):
        """Interleaves the x's and the y's into a single sequence."""
        bsize, points, dim = xs_b.shape
        ys_b_wide = torch.cat(
            (
                ys_b.view(bsize, points, 1),
                torch.zeros(bsize, points, dim - 1, device=ys_b.device),
            ),
            axis=2,
        )
        zs = torch.stack((xs_b, ys_b_wide), dim=2)
        zs = zs.view(bsize, 2 * points, dim)
        return zs

In [4]:
x = torch.tensor([[[1,2,3], [10,11,12]]])
y = torch.tensor([[7,8]])
print(_combine(x,y))

tensor([[[ 1.,  2.,  3.],
         [ 7.,  0.,  0.],
         [10., 11., 12.],
         [ 8.,  0.,  0.]]])


In [6]:
b_size = 2
n_dims = 10
n_points = 20
scale = 1
xs_b = torch.randn(b_size, n_points, n_dims)
w_b = torch.randn(b_size, n_dims, 1)
ys_b = scale * (xs_b @ w_b)[:, :, 0]
# print(xs_b)
# print(w_b)
ys_b

tensor([[ 4.2928,  0.7752, -2.0067, -2.7497, -2.2065,  1.6115, -2.3812, -1.0388,
          2.3826,  0.9312, -1.0446,  0.2875, -2.2057, -0.9914, -2.4912,  2.3056,
          1.5789,  0.7910,  1.9145, -0.0324],
        [ 0.2469,  1.3184, -2.2832,  2.8630, -0.3621, -0.6629,  0.7160,  2.6438,
          1.0135,  3.1089,  0.7629,  0.4742,  2.7834,  0.2663, -0.3894,  1.6009,
          2.6995,  2.0222, -0.8605, -3.5497]])

In [9]:
z_b = _combine(xs_b, ys_b)
z_b.shape

torch.Size([2, 40, 10])

In [11]:
xs_b

tensor([[[-0.8298, -0.1535, -0.5962,  0.3541, -0.8084,  2.4612, -0.4342,
           1.3519,  0.3254, -0.9109],
         [ 0.6092,  0.1813,  0.4335,  0.5475, -2.2686, -1.0203,  0.5689,
           1.2262, -0.2323, -1.8412],
         [ 0.3058, -1.4339, -0.2044, -2.0097,  0.8852,  0.6958,  1.1562,
          -0.8045,  0.1683,  0.8235],
         [ 0.5652, -1.3923, -0.3780,  0.0656,  0.1295, -1.5281, -1.6986,
           0.9713, -0.0170, -0.2620],
         [ 0.5552, -1.1020,  0.0711,  0.2145, -0.6065, -0.5450,  0.8142,
          -0.5801,  1.6797,  0.3534],
         [-0.3085, -0.5251,  0.2234,  0.5668,  0.1314,  0.4286, -1.0220,
          -1.1275, -0.4713, -0.4090],
         [-0.1098, -0.2533, -0.6604,  2.2849,  0.6167, -0.7686, -0.3152,
          -0.0256,  2.1612,  1.1536],
         [-0.6053,  1.4743,  1.3876, -2.3028,  0.6335, -1.3839, -0.3766,
           0.9980, -0.4045,  1.5799],
         [-0.3325, -0.3762,  0.2160,  0.7448, -0.6524,  0.8549, -0.9870,
          -1.4943,  0.8687,  1.0478],
 

In [13]:
z_b[0][1]

tensor([4.2928, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])